In [1]:
import os
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def visualization_tool(session, frame):
    # Initial setup & frame range
    pcd = o3d.geometry.PointCloud()
    current_frame = frame

    folder_path = f'sessions/frames_{session:02d}'
    entries = os.listdir(folder_path)
    max_frame = len(entries) - 1

    def plot_frame(session, frame):
        file_name = os.path.join(f'sessions/frames_{session:02d}', f'frame_{frame:05d}.npy')
        point_cloud_np = np.load(file_name)
        # Create an Open3D PointCloud object
        #point_cloud_o3d = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(point_cloud_np)
        
        #o3d.visualization.draw_geometries([point_cloud_o3d])
        return pcd
    
    # Key callback function for moving to the next frame
    def next_frame(vis):
        nonlocal current_frame
        nonlocal max_frame
        if current_frame < max_frame:
            current_frame += 1
            plot_frame(session, current_frame)
            vis.update_geometry(pcd)
            vis.poll_events()
            vis.update_renderer()

    # Key callback function for moving to the previous frame
    def prev_frame(vis):
        nonlocal current_frame
        if current_frame > 0:
            current_frame -= 1
            plot_frame(session, current_frame)
            vis.update_geometry(pcd)
            vis.poll_events()
            vis.update_renderer()
    
    # Create an Open3D visualization object
    vis = o3d.visualization.VisualizerWithKeyCallback()
    vis.create_window()

    # Add the initial frame's point cloud to the visualization
    vis.add_geometry(plot_frame(session, current_frame))

    # Register key callbacks for next and previous frames
    vis.register_key_callback(262, next_frame)
    vis.register_key_callback(263, prev_frame)

    # Start/Stop the visualization loop
    vis.run()
    vis.destroy_window()

In [3]:
visualization_tool(0,0)